# a5 - Tweeter Sentiment

In this assignment you will write a program to perform simple [sentiment analysis](https://en.wikipedia.org/wiki/Sentiment_analysis) of Twitter data&mdash;that is, determining the "attitude" or "emotion" (e.g., how "positive", "negative", "joyful", etc) of tweets made by a particular Twitter user. Sentiment analysis is a fascinating field: researchers have shown that the "mood" of Twitter communication [reflects biological rhythms](http://www.nytimes.com/2011/09/30/science/30twitter.html) and can even be used to [predict the stock market](http://arxiv.org/pdf/1010.3003&embedded=true). The particular analysis you'll be performing is inspired by an investigation of [personal vs. organizational tweets](http://varianceexplained.org/r/trump-tweets/) (which has become less amusing over time).

You will be implementing a Python program that performs this analysis on **real data** taken directly from a Twitter user's timeline. In the end, your script will produce output similar to the following:

```
EMOTION       % WORDS  EXAMPLE WORDS                     HASHTAGS
positive      6.16%    learn, faculty, happy             #accesstoinfoday, #indigenouspeoplesday, #idealistfair
trust         3.08%    school, faculty, happy            #indigenouspeoplesday, #diversity
anticipation  2.53%    happy, top, ready                 #indigenouspeoplesday, #informatics, #info340
joy           1.76%    happy, peace, deal                #indigenouspeoplesday, #accesstoinfoday
surprise      0.99%    deal, award, surprised            #suzzallolibrary, #nobrainer
negative      0.88%    fall, rejection, outstanding        
sadness       0.55%    fall, rejection, problem            
disgust       0.44%    rejection, weird, finally           
fear          0.44%    rejection, surprise, problem        
anger         0.33%    rejection, disaster, involvement  #mlis
```

Fill in the below code cells as specified. Note that cells may utilize variables and functions defined in previous cells; we should be able to use the `Kernal > Restart & Clear All` menu item followed by `Cell > Run All` to execute your entire notebook and see the correct output.

## The Data
You'll be working with two different pieces of data for this assignment.

First, you'll be loading tweet data taken directly from [Twitter's API](https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-user_timeline). You can find an example of this tweet data in the **`uw_ischool_sample.py`** file inside the `data/` folder. The below cell will import this data as a variable `SAMPLE_TWEETS` from the provided _module_ file:

In [1]:
# import from uw_ischool_sample file in the `data/` package (folder)
from data.uw_ischool_sample import SAMPLE_TWEETS

The data is represented as one giant **list of dictionaries**: the **list** contains a sequence of **dictionaries**, where each dictionary represents a tweet. Each dictionary contains many different _value_, some of which themselves may be dictionaries.

Print out the first three elements from the `SAMPLE_TWEETS` list to see what information can be found. The most relevant value is the `"text"` of the tweet.
- The Twitter API actually provides a lot more information about each tweet; I've stripped it down to only the most important properties for readability. Each dictionary is a proper subset of the full data you'd get from Twitter.
- Because of the source of the sentiment data, your analysis will be biased and only support English-language speakers. Nevertheless, Twitter is an international community so you may encounter non-English characters and words. You'll be working with real-world data and it will be messy!

In [2]:
SAMPLE_TWEETS[:3]

[{'created_at': 'Mon Oct 10 18:39:51 +0000 2016',
  'retweet_count': 9,
  'entities': {'hashtags': [{'indices': [20, 41],
     'text': 'IndigenousPeoplesDay'}]},
  'user': {'screen_name': 'UW_iSchool'},
  'text': 'RT @UWAPress: Happy #IndigenousPeoplesDay https://t.co/YmU9e9lj7v'},
 {'created_at': 'Mon Oct 10 18:00:00 +0000 2016',
  'retweet_count': 0,
  'entities': {'hashtags': [{'indices': [16, 29], 'text': 'IdealistFair'}]},
  'user': {'screen_name': 'UW_iSchool'},
  'text': "We'll be at the #IdealistFair this evening on the Seattle U. campus. Come and learn about our graduate programs: https://t.co/et1HrQshmr"},
 {'created_at': 'Mon Oct 10 15:10:36 +0000 2016',
  'retweet_count': 1,
  'entities': {'hashtags': []},
  'user': {'screen_name': 'UW_iSchool'},
  'text': 'RT @iYouthUW: iYouth Tips for 1st\xa0Years https://t.co/K4SCIEhJ8k https://t.co/p4lbC6Jb5o'}]

The second piece of data you'll be working with is a set of **word-sentiments**&mdash;a list of English-language words and what emotions (e.g., "joy", "anger") [are associated with them](http://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm).

- The [`nltk`](https://github.com/nltk/nltk/wiki/Sentiment-Analysis) library you used in the last assignment does support sentiment analysis. However, for practice and extendability you'll be doing a more "manual" analysis using the provided data file for this assignment.

`import` the word sentiments as a variable **`SENTIMENTS`** from the **`data.sentiments_nrc`** module. You should also import the `EMOTIONS` variable provided by the same module: this is a _list_ of possible emotions. You can inspect the variables (e.g., print them out) to confirm that you have imported them.

In [3]:
from data.sentiments_nrc import SENTIMENTS, EMOTIONS
print(EMOTIONS)
SENTIMENTS

['positive', 'negative', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust']


{'abacus': {'trust': 1},
 'abandon': {'sadness': 1, 'fear': 1, 'negative': 1},
 'abandoned': {'sadness': 1, 'negative': 1, 'anger': 1, 'fear': 1},
 'abandonment': {'surprise': 1,
  'sadness': 1,
  'negative': 1,
  'anger': 1,
  'fear': 1},
 'abba': {'positive': 1},
 'abbot': {'trust': 1},
 'abduction': {'sadness': 1, 'surprise': 1, 'fear': 1, 'negative': 1},
 'aberrant': {'negative': 1},
 'aberration': {'disgust': 1, 'negative': 1},
 'abhor': {'disgust': 1, 'negative': 1, 'anger': 1, 'fear': 1},
 'abhorrent': {'disgust': 1, 'negative': 1, 'anger': 1, 'fear': 1},
 'ability': {'positive': 1},
 'abject': {'disgust': 1, 'negative': 1},
 'abnormal': {'disgust': 1, 'negative': 1},
 'abolish': {'anger': 1, 'negative': 1},
 'abolition': {'negative': 1},
 'abominable': {'disgust': 1, 'fear': 1, 'negative': 1},
 'abomination': {'disgust': 1, 'negative': 1, 'anger': 1, 'fear': 1},
 'abort': {'negative': 1},
 'abortion': {'sadness': 1, 'disgust': 1, 'fear': 1, 'negative': 1},
 'abortive': {'sadnes

## Text Sentiment
All of the sentiment analysis is based on the individual _words_ in the text. Thus you will need to will determine which words in a tweet have which sentiments.

- Note that the assignment explicitly does _not_ tell you what to name functions, what arguments they should take or values they should return: your task is to determine appropriate functions and arguments from the (guided) requirements! Use multiple functions for clarity, give them all informative names, and include a **doc string** to explain what it does.

Define a function that take a tweet's text (a string) and split it up into a list of individual words.

- To support future assignments, you should **not** use the `nltk` library to tokenize words. Instead, your analysis should split up the text using the [regular expression](https://www.regular-expressions.info/) **`"\W+"`** as a separator to "split up" the words by (rather than just a blank space). You can do this by using the [re.split()](https://docs.python.org/3/library/re.html#re.split) function (from the `re` module). This separator will cause your spitting to exclude punctuation and provide a reasonable (but not perfect!) list of words to consider. 

- All of the words in the sentiment dictionary are _lower case_, so you'll need to **map** your resulting words to be lower case. You will also need to **filter** out any words that have 1 letter or fewer. Use a **list comprehension** to do this.

The string `"Amazingly, I prefer a #rainy day to #sunshine."` should produce a list with 6 lower-case words in it.

In [4]:
def split_words(text):
    """Take in a string as "text", then transform the string into lower case, split up and store into a list(split_text)"""
    import re
    text = text.lower()
    split_text = re.split(r'\W+',text)
    for words in split_text:
        if len(words) <=1:
            split_text.remove(words)
    return(split_text)
split_words("Amazingly, I prefer a #rainy day to #sunshine.")

['amazingly', 'prefer', 'rainy', 'day', 'to', 'sunshine']

Define a function that **filters** a list of the words to get only those words that contain a specific emotion. Use a **list comprehension** to do this.
- You can determine whether a word has a particular emotion by looking it up in the imported `SENTIMENTS` variable. Use the word as the "key" to find the dictionary of emotions for that word, and then use the emotion as the key to _that_ dictionary to determine if the word has it. 
    
    Do not use the `in` operator or a loop to search the list of keys one by one. Instead, use bracket notation or the `get()` method to "look up" a key directly (this is more efficient and is why the emotions are nested dictionaries in the first place).
    
For testing, the `"positive"` words extracted from `"Amazingly, I prefer a #rainy day to #sunshine."` are `["amazingly", "prefer", "sunshine"]`.

In [5]:
# I'm not sure if I understand the prompt correctly... 
def filter_emotions(text):
    """Take in a list of text and filter out words without sentiments. Output sentiments words as a list(emotions_text)"""
    emotions_text = []
    for words in text:
        if SENTIMENTS.get(words,"n") != "n":
            emotions_text.append(words)
    return(emotions_text)
filter_emotions(split_words("Amazingly, I prefer a #rainy day to #sunshine."))

['amazingly', 'prefer', 'rainy', 'sunshine']

Define a function that determines which words from a list have _each_ emotion (i.e., the "emotional" words). For example, the words extracted from `"Amazingly, I prefer a #rainy day to #sunshine."` should produce a dictionary that looks like:

```
{
 'anger': [],
 'anticipation': [],
 'disgust': [],
 'fear': [],
 'joy': ['amazingly', 'sunshine'],
 'negative': [],
 'positive': ['amazingly', 'prefer', 'sunshine'],
 'sadness': ['rainy'],
 'surprise': ['amazingly'],
 'trust': ['prefer']
}
```
    
(Note the empty lists for emotions that have no matching words).
    
- You can use the imported `EMOTIONS` variable to have a list of emotions to iterate through.
- Use the function you defined in the previous step to help you out!
- Using a [dictionary comprehension](https://www.smallsurething.com/list-dict-and-set-comprehensions-by-example/) is a nice way to do this, but is not required.

In [6]:
def emotions_sort(text):
    """Take in a list of text that contain sentiments, and output a dictionary categorizing words with associated emotions."""
    emotions_text = filter_emotions(text)
    emotions_dict = {}
    for i in sorted(EMOTIONS):
        emotions_dict[i]=[]
    for words in emotions_text:
        for i in SENTIMENTS.get(words):
            emotions_dict[i].append(words)
    return(emotions_dict)
emotions_sort(split_words("Amazingly, I prefer a #rainy day to #sunshine."))

{'anger': [],
 'anticipation': [],
 'disgust': [],
 'fear': [],
 'joy': ['amazingly', 'sunshine'],
 'negative': [],
 'positive': ['amazingly', 'prefer', 'sunshine'],
 'sadness': ['rainy'],
 'surprise': ['amazingly'],
 'trust': ['prefer']}

Define a function that gets a list of the "most common" words in a list: that is a new list containing each word in the original list, in descending order by how many times that word appears in the orignal list.

- You can determine the frequency (number of occurrences) of a word with a similar process to what you did with digits in the last assignment.
- You should use the `sorted()` function to [sort](https://wiki.python.org/moin/HowTo/Sorting) the individual words. This function take a **`key`** argument which should be passed a [_callback function_](https://wiki.python.org/moin/HowTo/Sorting#Key_Functions) that can return a "transformed" value that you wish to sort by (e.g., which element in a tuple). An anonymous lambda function works well for this.

You can test this function with any list of "words" with repeated entries: `['a','b','c','c','c','a']` for example.


In [7]:
def most_common(wordlist):
    """Take in a list of words and find the top 3 common words in the list. Output as a list(most_common)"""
    text_count = {}
    for words in wordlist:
        prev = text_count.get(words,0)
        text_count[words] = prev + 1
    text_count_list = []
    for i in text_count:
        text_count_list.append((i,text_count[i]))
    text_count_list = sorted(text_count_list, key=lambda count:count[1], reverse=True)
#     In order to make it easier to use in the following functions, transform the tuples into list of common words. 
    most_common = []
    for a in text_count_list:
        most_common.append(a[0])
    return(most_common)
most_common(['a','b','c','c','c','a'])

['c', 'a', 'b']

## Tweet Statistics
Once you are able to determine the sentiment of an individual string of text (e.g., a single tweet's content), you can analyze an entire set of tweets from the user's timeline.

Define a function (e.g., `analyze_tweets()`) that takes as an argument a **list** of tweet data (with the same structure as the imported `SAMPLE_TWEETS` variable), and _returns_ the data of interest to display in a table like the one at the very top of the notebook. In particular, you'll need to produce the following information **for each emotion**:

1. The percentage of words _across all tweets_ that have that emotion
2. The most common words _across all tweets_ that have that emotion (in order!)
3. The most common **hashtags** _across all tweets_ associated with that emotion (see below)

(Think carefully: should this data be stored in a _list_ or a _dictionary_?)

Some tips for this task:

- You can optionally create some "helper" functions to break up this task even further; define those functions in the same notebook cell or add additional cells.

- You'll need to use your previous functions to get the _list of words_ and _dictionary of emotional words_ for each tweet. I recommend you assign the results of these methods as **new keys** of the respective tweet dictionary (so your tweet would gain a `words` key, for example).

- In order to get the percentage of emotional words, divide the number of words that have that emotion by the total number of words _across all the tweets_. Counting how many total words are in the tweet set is a **reducing** operation: you should use the `reduce()` function for this.

- For each emotion, you'll need to get a list of the words _across all the tweets_ that have that emotion (in order to determine how many there are for the percentage, as well as which are most common). This is another **reducing** operation; you should use the `reduce()` function to _add up_ all of these words (alternatively, the `sum()` function can be used here).

- For emotion emotion, you will also need to calculate the most common [hashtags](https://en.wikipedia.org/wiki/Hashtag) for tweets that have _at least one word with that emotion_.

    The Twitter data for each tweet includes a _list_ containing the hashtags found in that tweet&mdash;you should **NOT** try and search the tweet text for `#` symbols. These hashtags can be found in the `['entities']['hashtags'][i]['text']` element of each tweet&mdash;that is, the `'text'` key from _each_ element in the _list_ of the `'hashtags'` key in the `'entities'` dictionary of the tweet. See the `uw_school.json` example file to see this structure more clearly.

    (You might use a _list comprehension_ to "flatten" this complex nesting structure into just a list of hashtag words).

    Since hashtags are just words, you can use your function for finding the most common words to find the most common hashtags!

You can test your function by passing in the `SAMPLE_TWEETS` variable as an argument and checking if your returned data has the same numbers as in the table at the top of the page. Note that only the first 3 most common words are listed (and may be in a different order in the case of ties).

In [8]:
def analyze_tweets(tweet):
    """Take in a list of tweet data and output the percentages of sentiments words, most common words and hashtags associated with that emotion."""
    import re
    from functools import reduce
    # Create a dictionary to store emotion words and hashtags based on the sentiments associated.
    emotions_dict = {'anger': [], 'anticipation': [], 'disgust': [], 'fear': [], 'joy': [], 'negative': [], 'positive': [], 'sadness': [], 'surprise': [], 'trust': []}
    hashtags_dict = {'anger': [], 'anticipation': [], 'disgust': [], 'fear': [], 'joy': [], 'negative': [], 'positive': [], 'sadness': [], 'surprise': [], 'trust': []}
    for i in range(len(tweet)):
        # My number of percentages are different from the sample shown above.
        # I'm not sure if it is because I split the text using "space" or other reasons.
        split_text = re.split(" ",tweet[i]['text'])
        # Create keys in the orignal data to store the value of each analysis, including split text, number of words in each tweet, emotions associated, and hashtags.
        tweet[i]['split_text'] = split_text
        tweet[i]['word_cnt'] = len(split_text)
        tweet[i]['emotions'] = emotions_sort(split_words(tweet[i]['text']))
        tweet[i]['hashtags'] = []
        for a in range(len(tweet[i]['entities']['hashtags'])):
            tweet[i]['hashtags'].append(tweet[i]['entities']['hashtags'][a]['text'].lower())
        for emotion in emotions_dict:
            emotions_dict[emotion] = emotions_dict[emotion] + tweet[i]['emotions'][emotion]
            # Categorize the hashtags by emotions.
            if len(tweet[i]['hashtags'])>0 and len(tweet[i]['emotions'][emotion])>0:
                hashtags_dict[emotion] = hashtags_dict[emotion] + tweet[i]['hashtags']
    most_common_emotion_word = {}
    most_common_hashtags = {}
    emotions_count = []
    # Count the total number of words across the tweets.
    word_count = 0
    total_count = reduce(lambda word_count , new : word_count + new['word_cnt'], tweet, 0)
    # Get the most common words and hashtags in each emotions.
    for emotion in EMOTIONS:
        most_common_emotion_word[emotion] = most_common(emotions_dict[emotion])[:3]
        most_common_hashtags[emotion] = most_common(hashtags_dict[emotion])[:3]
        temp = (emotion, round(len(emotions_dict[emotion])/total_count*100,2))
        emotions_count.append(temp)
    # Sort the emotions by percentages.
    emotions_count = sorted(emotions_count, key=lambda count:count[1], reverse=True)
    return(emotions_count, most_common_emotion_word, most_common_hashtags)
analyze_tweets(SAMPLE_TWEETS)

([('positive', 6.72),
  ('trust', 3.36),
  ('anticipation', 2.76),
  ('joy', 1.92),
  ('surprise', 1.08),
  ('negative', 0.96),
  ('sadness', 0.6),
  ('disgust', 0.48),
  ('fear', 0.48),
  ('anger', 0.36)],
 {'positive': ['learn', 'faculty', 'happy'],
  'negative': ['fall', 'rejection', 'outstanding'],
  'anger': ['rejection', 'disaster', 'involvement'],
  'anticipation': ['happy', 'top', 'ready'],
  'disgust': ['rejection', 'weird', 'finally'],
  'fear': ['rejection', 'surprise', 'problem'],
  'joy': ['happy', 'peace', 'deal'],
  'sadness': ['fall', 'rejection', 'problem'],
  'surprise': ['deal', 'award', 'surprised'],
  'trust': ['school', 'faculty', 'happy']},
 {'positive': ['accesstoinfoday', 'indigenouspeoplesday', 'idealistfair'],
  'negative': [],
  'anger': ['mlis'],
  'anticipation': ['indigenouspeoplesday', 'informatics', 'info340'],
  'disgust': [],
  'fear': [],
  'joy': ['indigenouspeoplesday', 'accesstoinfoday'],
  'sadness': [],
  'surprise': ['suzzallolibrary', 'nobrain

Once you've analyzed the tweets, you will need to _display_ that information as a printed table (as in the example at the top of the page).

Define another function to display this table (your function should take as an argument the data structure returned from your "analysis" function).

This function will need to print out the table. Using the [string formatting](https://docs.python.org/3/library/string.html#format-examples) language (via the **`.format()`** string method) makes it possible to have equally sized "columns" of data. For more example, [this tutorial](https://www.digitalocean.com/community/tutorials/how-to-use-string-formatters-in-python-3) is pretty good (check out the "Padding Variable Substitutions" section).


A few notes about formatting this output:

- For your reference, the example table at the top of the page uses `14` characters for the first column, `11` characters for the second,  `35` for the third, and the "remainder" for the fourth. You are not required to match these numbers.

- The percentage should be formatted with two decimals of precision (e.g., `1.23%`).

- Both the example sentiment words and the hashtags should be outputted as a _comma-separated list_ with spaces between them (and no square brackets). The `join()` string method is good for converting lists to formatted strings. Both lists should also be limited to the 3 most common items.

- Make sure to include `#` in front of the hashtags!

In [9]:
def format_table(analysis):
    """Print out a table to show the percentages of each emotions, the most common words and hashtags of that emotion."""
    # Header of the table
    header = ['EMOTION', '% WORDS', 'EXAMPLE WORDS', 'HASHTAGS']
    print("{:<15} {:<10} {:<35} {}".format(*header))
    comma = ", "
    hashtag = ", #"
    # Get the information needed from the input data
    for i in range(len(EMOTIONS)): 
        emotion = analysis[0][i][0]
        percentage = str(analysis[0][i][1])+"%"
        words = comma.join(analysis[1][emotion])
        if len(analysis[2][emotion])> 0:
            hashtags = "#"+ hashtag.join(analysis[2][emotion])
        else:
            hashtags = ""
        print("{:<15} {:<10} {:<35} {}".format(emotion, percentage, words, hashtags))
format_table(analyze_tweets(SAMPLE_TWEETS))

EMOTION         % WORDS    EXAMPLE WORDS                       HASHTAGS
positive        6.72%      learn, faculty, happy               #accesstoinfoday, #indigenouspeoplesday, #idealistfair
trust           3.36%      school, faculty, happy              #indigenouspeoplesday, #diversity
anticipation    2.76%      happy, top, ready                   #indigenouspeoplesday, #informatics, #info340
joy             1.92%      happy, peace, deal                  #indigenouspeoplesday, #accesstoinfoday
surprise        1.08%      deal, award, surprised              #suzzallolibrary, #nobrainer
negative        0.96%      fall, rejection, outstanding        
sadness         0.6%       fall, rejection, problem            
disgust         0.48%      rejection, weird, finally           
fear            0.48%      rejection, surprise, problem        
anger           0.36%      rejection, disaster, involvement    #mlis


## Getting Live Data
This is all good and well, but the real payoff would be to be able to see the sentiments of tweets taken directly from the Twitter feed of real users!

Define _another_ function that takes in a Twitter username as an argument and then returns a list of dictionaries representing the tweets made by that user.

Normally you would fetch this data by sending a request directly to the web service's API (e.g., to the the [statuses/user_timeline](https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-user_timeline) endpoint provided by the Twitter API at `https://api.twitter.com/1.1/statuses/user_timeline`). However, Twitter includes access controls so that only registered developers are allowed to send requests. While it is possible to register as a developer and access Twitter [directly through Python](https://python-twitter.readthedocs.io/en/latest/), this adds an extra level of complexity to the assignment.

Instead, I've set up a [proxy](https://en.wikipedia.org/wiki/Proxy) that has all the access keys specified which you can use to search Twitter. This proxy is available at:

**<https://faculty.washington.edu/joelross/proxy/twitter/timeline/>**

Send a request to _that_ url instead of `https://api.twitter.com/1.1/statuses/user_timeline`, and it will redirect your request with the proper authentication to Twitter, and then give you back whatever JSON Twitter's API responded with.

- You specify the same request parameters as you would when accessing Twitter directly. The request takes a `screen_name` request parameter which you can assign the given username. You can also specify the `count` parameter if you want to get more results back (up to 200); see the [documentation](https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-user_timeline) for details and other options you are welcome to use (just document them in your function's **docstring**).

- **WARNING:** The proxy I have set up is **rate-limited** so that it can only accept 900 requests every 15 minutes. If all 40 students are working rapidly on the assignment at the same time, you may find yourself needing to wait a few minutes and try again. You are alternatively welcome to set up your own developer account and API keys; just make sure you don't put the keys under version control and upload them to GitHub!

You can download the timeline data from Twitter using the [requests](http://docs.python-requests.org/en/master/user/quickstart/) module discussed in class: send a `GET` request to the [statuses/user_timeline](https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-user_timeline) endpoint provided by the Twitter API, and then use the `.json()` method to extract the JSON response as a Python _list_ or _dictionary_ value you can work with.

In [10]:
def get_user(username):
    """Take in a Twitter username and output a structured tweet data of that account."""
    import requests, json
    query_params = {"screen_name": username}
    response = requests.get("https://faculty.washington.edu/joelross/proxy/twitter/timeline/", params = query_params)
    response_data = response.text
    text_data = json.loads(response_data)
    return(text_data)

Define one last "main" function that will [prompt the user](https://docs.python.org/3/library/functions.html#input) for a Twitter username. The function should then call your "download" function to fetch the tweets, and pass the returned tweet data into your "analyze" and "show" functions in order to display your sentiment analysis of the user's timeline. 

**ADDITIONALLY**, `if` the user specifies `SAMPLE` (all caps) as the username, the function should instead show the analysis for the `SAMPLE_TWEETS` (this will help us out with grading).

In [11]:
if __name__ == "__main__":
    # execute only if run as a script
    # Import emotions and sentiments data. Also import sample data.
    from data.uw_ischool_sample import SAMPLE_TWEETS
    from data.sentiments_nrc import SENTIMENTS, EMOTIONS
    username = input("Which Twitter account do you want to analyze? ")
    if username == "SAMPLE":
        tweet = SAMPLE_TWEETS
    else:
        tweet = get_user(username)
    analysis = analyze_tweets(tweet)
    format_table(analysis)

Which Twitter account do you want to analyze? Dior
EMOTION         % WORDS    EXAMPLE WORDS                       HASHTAGS
positive        8.18%      creative, lover, passion            #diormagazine, #peterphilips, #miwakomatsu
trust           3.69%      lover, passion, dance               #diormagazine, #peterphilips, #miwakomatsu
joy             3.43%      lover, passion, love                #miwakomatsu, #yoonahn, #missdior
anticipation    2.64%      lover, passion, calls               #diormagazine, #miwakomatsu, #yoonahn
negative        1.32%      shot, calls, frenzied               #diormagazine, #missdior
surprise        1.06%      inspired, shot, celebration         #diormagazine, #missdior, #peterphilips
fear            0.79%      shot, frenzied, whirlwind           #missdior, #diormagazine
anger           0.53%      shot, frenzied                      #diormagazine, #missdior
sadness         0.26%      shot                                #diormagazine
disgust         0.0%   

Use your main function to try analyzing the timelines of different users and comparing their results. Are the current sentiments of the [iSchool](https://twitter.com/uw_ischool) and [CSE](https://twitter.com/uwcse) different in interesting ways?

In [12]:
# The sentiments of iSchool and CSE are similar. 
# Their posts mostly show positive, trust, and anticipation. 
# The words they used might be different; however, they both often uses words related to faculty, students or friends.
# From the hashtags they used can tell their current focuses are different. CSE focuses more on AI, and iShool focuses on digital related articles.